## note: should be run with python3.6 to be able to use MTK library

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# import MisrToolkit as mtk # needs python3.6
# from mpl_toolkits.basemap import Basemap # needs python 3.8?
import sys # to check python version
import os
import glob

In [2]:
# check python version
sys.version

'3.6.9 (default, Dec  8 2021, 21:08:43) \n[GCC 8.4.0]'

In [3]:
# load training dataset
# ds_fp = "/Users/ehsanmos/Documents/RnD/MISR_lab/ML_research/training_dataset/merged_april_2016_9cam3bands.csv"
ds_fp = "/home/ehsan/misr_lab/training_dataset/april_2016/april_2016_parts/merged_april_2016_9cam_4bands_final_dataset_2.csv"
in_ds = pd.read_csv(ds_fp)

In [4]:
in_ds.head(5)

,path,orbit,block,line,sample,lat,lon,Da_r,Ca_r,Ba_r,Aa_r,An_r,An_g,An_b,An_nir,Af_r,Bf_r,Cf_r,Df_r,mean_ATM_roughness
0,0,0,0,0.00,0.00,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00
1,152,86922,9,75.15,1205.60,74.422639,215.545163,1.03203,0.84397,0.71459,0.62433,0.59725,0.47642,0.76723,-99.0,0.70105,0.92672,1.39007,2.15582,5.19
2,152,86922,9,74.95,1205.52,74.422351,215.546735,1.03353,0.83645,0.71008,0.61530,0.58973,0.48393,0.76573,-99.0,0.69955,0.94627,1.39609,2.15582,5.72
3,152,86922,9,74.76,1205.45,74.422064,215.548307,1.03353,0.83645,0.71008,0.61530,0.58973,0.48393,0.76573,-99.0,0.69955,0.94627,1.39609,2.15582,6.33
4,152,86922,9,74.92,1205.51,74.422294,215.547049,1.03353,0.83645,0.71008,0.61530,0.58973,0.48393,0.76573,-99.0,0.69955,0.94627,1.39609,2.15582,5.97


In [5]:
in_ds.shape

(2924469, 20)

In [6]:
ds_3cam = in_ds[["path", "orbit", "block", "line", "sample", "lat", "lon", "Ca_r", "Cf_r", "An_r", "mean_ATM_roughness"]] # add path & block here


In [7]:
ds_3cam.rename(columns={"mean_ATM_roughness":"rms"}, inplace=True)
# ds_3cam["mean_ATM_roughness"]="rms" # does not work- should use DF.rename()
ds_3cam.columns

/home/ehsan/.local/lib/python3.6/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Index(['path', 'orbit', 'block', 'line', 'sample', 'lat', 'lon', 'Ca_r',
       'Cf_r', 'An_r', 'rms'],
      dtype='object')

In [8]:
# determine range to print based on min, max lat and lon of the data
# is it necessary?
xlon = ds_3cam['lon']
ylat = ds_3cam['lat']


margin = 2 # buffer to add to the range
lat_min = min(ylat) - margin
lat_max = max(ylat) + margin
lon_min = min(xlon) - margin
lon_max = max(xlon) + margin

In [9]:
print(lat_min)
print(lat_max)
print(lon_min)
print(lon_max)

-2.0
86.250712
-2.0
277.113721


In [10]:
from itertools import chain

def draw_map(m, scale=0.2):
    # draw a shaded-relief image
    m.shadedrelief(scale=scale)
    
    # lats and longs are returned as a dictionary
    lats = m.drawparallels(np.linspace(-90, 90, num=7))
    lons = m.drawmeridians(np.linspace(-180, 180, num=18))

    # keys contain the plt.Line2D instances
    lat_lines = chain(*(tup[1][0] for tup in lats.items()))
    lon_lines = chain(*(tup[1][0] for tup in lons.items()))
    all_lines = chain(lat_lines, lon_lines)
    
    # cycle through these lines and set the desired style
    for line in all_lines:
        line.set(linestyle='-', alpha=0.3, color='w')

In [11]:
# ref: https://stackoverflow.com/questions/44488167/plotting-lat-long-points-using-basemap
# basemap = Basemap(llcrnrlon=lon_min,
#                     llcrnrlat=lat_min,
#                     urcrnrlon=lon_max,
#                     urcrnrlat=lat_max,
#                     lat_0=(lat_max - lat_min)/2,
#                     lon_0=(lon_max-lon_min)/2,
#                     projection='ortho',
#                     resolution = None,
#                     area_thresh=10000.,)

In [12]:
# basemap.shadedrelief()
# basemap.drawcoastlines(color='gray')
# basemap.drawcountries(color='gray')
# basemap.drawstates(color='gray')



# basemap.drawcoastlines()
# basemap.drawcountries()
# basemap.drawstates()
# basemap.drawmapboundary(fill_color='#46bcec')
# basemap.fillcontinents(color = 'white',lake_color='#46bcec')

In [13]:
# fig = plt.figure(figsize=(12, 12))

# basemap = Basemap(projection='ortho', resolution=None, lat_0=90, lon_0=180)

# # locations on map: convert lat and lon to map projection coordinates
# xlons, ylats = basemap(xlon, ylat)

# plt.plot(xlons, ylats, 'ok', markersize=1)

# # draw a circle on map on 60N  
# basemap.drawparallels([60], color='red') #, labels='test'????)
# plt.title('Study Region, 60N', fontsize=16)

# draw_map(basemap);

# how zoom to the region?

In [14]:
def make_the_map(XlonSet, YlatSet):
    
    fig = plt.figure(figsize=(12, 12))

    basemap = Basemap(projection='ortho', resolution=None, lat_0=90, lon_0=180)

    # locations on map: convert lat and lon to map projection coordinates
    xlons, ylats = basemap(XlonSet, YlatSet)

    plt.plot(xlons, ylats, 'ok', markersize=1)

    # draw a circle on map on 60N  
    basemap.drawparallels([60], color='red') #, labels='test'????)
    plt.title('Study Region, 60N', fontsize=16)

    draw_map(basemap);
    return 0;

In [15]:
# run with oython >= 3.8

# make_the_map(xlon, ylat)

In [16]:
# extract transect
transect = ds_3cam.loc[(ds_3cam['lon'] > 200.0) & (ds_3cam['lon'] < 230.0)]
transect

,path,orbit,block,line,sample,lat,lon,Ca_r,Cf_r,An_r,rms
1,152,86922,9,75.15,1205.60,74.422639,215.545163,0.84397,1.39007,0.59725,5.19
2,152,86922,9,74.95,1205.52,74.422351,215.546735,0.83645,1.39609,0.58973,5.72
3,152,86922,9,74.76,1205.45,74.422064,215.548307,0.83645,1.39609,0.58973,6.33
4,152,86922,9,74.92,1205.51,74.422294,215.547049,0.83645,1.39609,0.58973,5.97
5,152,86922,9,75.10,1205.71,74.422393,215.544545,0.84397,1.39007,0.59725,5.18
...,...,...,...,...,...,...,...,...,...,...,...
2924464,86,86947,31,248.73,1902.06,71.759078,203.910947,-1.00000,-1.00000,-1.00000,17.96
2924465,86,86947,31,248.84,1902.09,71.758824,203.910650,-1.00000,-1.00000,-1.00000,26.55
2924466,86,86947,31,248.65,1902.21,71.759035,203.912238,-1.00000,-1.00000,-1.00000,16.72
2924467,86,86947,31,248.47,1902.33,71.759246,203.913826,-1.00000,-1.00000,-1.00000,10.74


In [17]:
# create a column as index and set that column as index to plot transect

# lon_subset['index'] = [i for i in range(len(lon_subset['lon']))]
transect.index = range(1,len(transect)+1)
transect.head()

# set that column as index
# lon_subset.set_index['index']

,path,orbit,block,line,sample,lat,lon,Ca_r,Cf_r,An_r,rms
1,152,86922,9,75.15,1205.60,74.422639,215.545163,0.84397,1.39007,0.59725,5.19
2,152,86922,9,74.95,1205.52,74.422351,215.546735,0.83645,1.39609,0.58973,5.72
3,152,86922,9,74.76,1205.45,74.422064,215.548307,0.83645,1.39609,0.58973,6.33
4,152,86922,9,74.92,1205.51,74.422294,215.547049,0.83645,1.39609,0.58973,5.97
5,152,86922,9,75.10,1205.71,74.422393,215.544545,0.84397,1.39007,0.59725,5.18


In [18]:
# select a smaller subset of transect, like the first 1000 points (or 10)
transect_small = transect.iloc[0:10000]
transect_small

,path,orbit,block,line,sample,lat,lon,Ca_r,Cf_r,An_r,rms
1,152,86922,9,75.15,1205.60,74.422639,215.545163,0.84397,1.39007,0.59725,5.19
2,152,86922,9,74.95,1205.52,74.422351,215.546735,0.83645,1.39609,0.58973,5.72
3,152,86922,9,74.76,1205.45,74.422064,215.548307,0.83645,1.39609,0.58973,6.33
4,152,86922,9,74.92,1205.51,74.422294,215.547049,0.83645,1.39609,0.58973,5.97
5,152,86922,9,75.10,1205.71,74.422393,215.544545,0.84397,1.39007,0.59725,5.18
...,...,...,...,...,...,...,...,...,...,...,...
9996,79,86932,29,417.91,1364.11,74.244492,215.095957,0.85789,0.96812,0.80035,6.02
9997,79,86932,29,417.90,1364.32,74.244203,215.097542,0.85789,0.96812,0.80035,7.34
9998,79,86932,29,417.91,1364.13,74.244466,215.096102,0.85789,0.96812,0.80035,6.13
9999,79,86932,29,418.03,1363.90,74.244545,215.093780,0.85494,0.96673,0.80104,7.45


In [19]:
xlon270 = transect_small['lon']
ylat270 = transect_small['lat']

In [22]:
# run with oython >= 3.8

# make_the_map(xlon270, ylat270)

In [20]:
# # extract transect data write it out

# # output_filename = "3cams_outofsample_atm.csv"
# output_filename = "insample_atm_withPathOrbit_3cams_forTransect.csv"

# # output_path = "/Users/ehsanmos/Documents/RnD/MISR_lab/ML_research/training_dataset"
# output_path = "/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/training_data"

# transect_small.to_csv(os.path.join(output_path, output_filename))